In [ ]:
!pip install opencv-python pyqt5

In [1]:
import sys
import os
import cv2
import numpy as np
from PyQt5.QtWidgets import (
    QApplication, QMainWindow, QWidget, QVBoxLayout, QHBoxLayout, QPushButton,
    QFileDialog, QLabel, QListWidget, QSlider, QMessageBox, QGroupBox, QGridLayout,
    QSpinBox, QScrollArea, QSplitter, QProgressBar, QCheckBox, QAction, QMenuBar,
    QDialog, QDialogButtonBox, QDoubleSpinBox
)
from PyQt5.QtGui import QPixmap, QImage, QPainter, QPen, QColor, QPalette
from PyQt5.QtCore import Qt, QRect, pyqtSignal

# Acceptable image file extensions.
IMAGE_EXTENSIONS = (".png", ".jpg", ".jpeg", ".bmp", ".tif", ".tiff", ".webp")


class ImageItem:
    """Holds image data and transformation parameters."""
    def __init__(self, path):
        self.path = path
        self.filename = os.path.basename(path)
        try:
            self.original_img = cv2.imread(path, cv2.IMREAD_UNCHANGED)
            if self.original_img is None:
                raise ValueError(f"Failed to load image: {path}")
            # Ensure image has an alpha channel.
            if len(self.original_img.shape) == 2:
                self.original_img = cv2.cvtColor(self.original_img, cv2.COLOR_GRAY2BGRA)
            elif self.original_img.shape[2] == 3:
                self.original_img = cv2.cvtColor(self.original_img, cv2.COLOR_BGR2BGRA)
            self.display_img = self.original_img.copy()
        except Exception as e:
            print(f"Error loading image {path}: {e}")
            self.original_img = None
            self.display_img = None

        self.rotation_angle = 0
        self.x_offset = 0
        self.y_offset = 0
        self.transparency = 255  # 0 (transparent) to 255 (opaque)
        self.crop_rect = None    # (x, y, w, h) in image coordinates
        self.transformed = False

    def reset(self):
        if self.original_img is not None:
            self.display_img = self.original_img.copy()
        self.rotation_angle = 0
        self.x_offset = 0
        self.y_offset = 0
        self.transparency = 255
        self.crop_rect = None
        self.transformed = False

    def get_cropped_image(self):
        if self.crop_rect is None or self.display_img is None:
            return self.display_img
        x, y, w, h = self.crop_rect
        x = max(0, min(x, self.display_img.shape[1] - 1))
        y = max(0, min(y, self.display_img.shape[0] - 1))
        w = min(w, self.display_img.shape[1] - x)
        h = min(h, self.display_img.shape[0] - y)
        return self.display_img[y:y+h, x:x+w]

    def is_valid(self):
        return self.original_img is not None


class ImagePreviewWidget(QWidget):
    """Displays the image preview and handles cropping."""
    cropChanged = pyqtSignal(QRect)

    def __init__(self, parent=None):
        super().__init__(parent)
        self.image = None
        self.display_pixmap = None
        self.bg_image = None  # Optional background image for previous layer
        self.crop_rect = None  # QRect in image coordinates
        self.is_dragging = False
        self.crop_mode = False
        self.checkerboard_bg = True
        self.scale_factor = 1.0
        self.offset = (0, 0)  # (x_offset, y_offset) for centering

        self.setMinimumSize(500, 400)
        self.setMouseTracking(True)

    def set_background_image(self, img):
        self.bg_image = img
        self.update()

    def toggle_checkerboard(self, enabled):
        self.checkerboard_bg = enabled
        self.update()

    def set_image(self, img):
        self.image = img
        if img is not None:
            height, width = img.shape[:2]
            bytes_per_line = 4 * width
            q_img = QImage(img.data, width, height, bytes_per_line, QImage.Format_RGBA8888).rgbSwapped()
            self.display_pixmap = QPixmap.fromImage(q_img)
        else:
            self.display_pixmap = None
        self.update_scale_and_offset()
        self.update()

    def update_scale_and_offset(self):
        if self.display_pixmap:
            w_img = self.display_pixmap.width()
            h_img = self.display_pixmap.height()
            scale_w = self.width() / w_img
            scale_h = self.height() / h_img
            self.scale_factor = min(scale_w, scale_h)
            new_w = w_img * self.scale_factor
            new_h = h_img * self.scale_factor
            self.offset = ((self.width() - new_w) / 2, (self.height() - new_h) / 2)
        else:
            self.scale_factor = 1.0
            self.offset = (0, 0)

    def resizeEvent(self, event):
        self.update_scale_and_offset()
        super().resizeEvent(event)

    def paintEvent(self, event):
        painter = QPainter(self)
        painter.fillRect(self.rect(), Qt.darkGray)
        if self.display_pixmap:
            x_off, y_off = self.offset
            target_rect = QRect(int(x_off), int(y_off),
                                int(self.display_pixmap.width() * self.scale_factor),
                                int(self.display_pixmap.height() * self.scale_factor))
            if self.bg_image is not None:
                h_bg, w_bg = self.bg_image.shape[:2]
                bytes_per_line = 4 * w_bg
                bg_qimg = QImage(self.bg_image.data, w_bg, h_bg, bytes_per_line, QImage.Format_RGBA8888).rgbSwapped()
                bg_pixmap = QPixmap.fromImage(bg_qimg)
                painter.drawPixmap(target_rect, bg_pixmap)
            elif self.checkerboard_bg:
                self.draw_checkerboard(painter, target_rect)
            painter.drawPixmap(target_rect, self.display_pixmap)
            if self.crop_rect:
                painter.setPen(QPen(Qt.red, 2, Qt.SolidLine))
                x = int(self.crop_rect.x() * self.scale_factor + x_off)
                y = int(self.crop_rect.y() * self.scale_factor + y_off)
                w = int(self.crop_rect.width() * self.scale_factor)
                h = int(self.crop_rect.height() * self.scale_factor)
                painter.drawRect(x, y, w, h)
                handle_size = 6
                painter.setBrush(Qt.red)
                painter.drawRect(x - handle_size//2, y - handle_size//2, handle_size, handle_size)
                painter.drawRect(x + w - handle_size//2, y - handle_size//2, handle_size, handle_size)
                painter.drawRect(x - handle_size//2, y + h - handle_size//2, handle_size, handle_size)
                painter.drawRect(x + w - handle_size//2, y + h - handle_size//2, handle_size, handle_size)

    def draw_checkerboard(self, painter, rect):
        checker_size = 10
        colors = [QColor(180, 180, 180), QColor(140, 140, 140)]
        for i in range(0, rect.width(), checker_size):
            for j in range(0, rect.height(), checker_size):
                color_index = (i // checker_size + j // checker_size) % 2
                painter.fillRect(rect.x() + i, rect.y() + j,
                                 min(checker_size, rect.width() - i),
                                 min(checker_size, rect.height() - j),
                                 colors[color_index])

    def mousePressEvent(self, event):
        if not self.display_pixmap or not self.crop_mode:
            return
        x_off, y_off = self.offset
        target_rect = QRect(int(x_off), int(y_off),
                            int(self.display_pixmap.width() * self.scale_factor),
                            int(self.display_pixmap.height() * self.scale_factor))
        if target_rect.contains(event.pos()):
            self.is_dragging = True
            pos = event.pos()
            x = int((pos.x() - x_off) / self.scale_factor)
            y = int((pos.y() - y_off) / self.scale_factor)
            self.crop_rect = QRect(x, y, 0, 0)
            self.update()

    def mouseMoveEvent(self, event):
        if self.is_dragging and self.crop_mode and self.crop_rect:
            x_off, y_off = self.offset
            pos = event.pos()
            current_x = int((pos.x() - x_off) / self.scale_factor)
            current_y = int((pos.y() - y_off) / self.scale_factor)
            start_x = self.crop_rect.x()
            start_y = self.crop_rect.y()
            width = current_x - start_x
            height = current_y - start_y
            if width < 0:
                start_x = current_x
                width = abs(width)
            if height < 0:
                start_y = current_y
                height = abs(height)
            self.crop_rect = QRect(start_x, start_y, width, height)
            self.update()

    def mouseReleaseEvent(self, event):
        if self.is_dragging and self.crop_mode:
            self.is_dragging = False
            if self.crop_rect and self.crop_rect.width() > 10 and self.crop_rect.height() > 10:
                if self.display_pixmap:
                    img_width = self.display_pixmap.width()
                    img_height = self.display_pixmap.height()
                    x = max(0, min(self.crop_rect.x(), img_width - 1))
                    y = max(0, min(self.crop_rect.y(), img_height - 1))
                    width = min(self.crop_rect.width(), img_width - x)
                    height = min(self.crop_rect.height(), img_height - y)
                    self.crop_rect = QRect(x, y, width, height)
                    self.cropChanged.emit(self.crop_rect)
                else:
                    self.crop_rect = None
            else:
                self.crop_rect = None
            self.update()

    def set_crop_mode(self, enabled):
        self.crop_mode = enabled
        self.update()


class CompareDialog(QDialog):
    """Dialog for comparing two images with adjustable blending."""
    def __init__(self, img1, img2, parent=None):
        super().__init__(parent)
        self.setWindowTitle("Compare Images")
        self.resize(900, 700)
        self.img1 = img1  # Bottom layer.
        self.img2 = img2  # Top layer.
        self.alpha = 0.5
        self.init_ui()

    def init_ui(self):
        layout = QVBoxLayout(self)
        title_label = QLabel("Image Comparison")
        title_label.setStyleSheet("font-weight: bold; font-size: 14px;")
        layout.addWidget(title_label)
        instruction_label = QLabel("Drag the slider to adjust blending between images")
        layout.addWidget(instruction_label)
        self.scroll_area = QScrollArea()
        self.scroll_area.setWidgetResizable(True)
        self.image_container = QWidget()
        self.image_label = QLabel()
        self.image_label.setAlignment(Qt.AlignCenter)
        container_layout = QVBoxLayout(self.image_container)
        container_layout.addWidget(self.image_label)
        container_layout.addStretch()
        self.scroll_area.setWidget(self.image_container)
        layout.addWidget(self.scroll_area, 1)
        slider_layout = QHBoxLayout()
        slider_layout.addWidget(QLabel("Image 1"))
        self.slider = QSlider(Qt.Horizontal)
        self.slider.setRange(0, 100)
        self.slider.setValue(50)
        self.slider.setTickPosition(QSlider.TicksBelow)
        self.slider.setTickInterval(10)
        self.slider.valueChanged.connect(self.update_blend)
        slider_layout.addWidget(self.slider, 1)
        slider_layout.addWidget(QLabel("Image 2"))
        layout.addLayout(slider_layout)
        self.blend_label = QLabel("Blend: 50% / 50%")
        self.blend_label.setAlignment(Qt.AlignCenter)
        layout.addWidget(self.blend_label)
        buttons = QDialogButtonBox(QDialogButtonBox.Close)
        buttons.rejected.connect(self.reject)
        layout.addWidget(buttons)
        self.update_blend()

    def update_blend(self):
        self.alpha = self.slider.value() / 100.0
        self.blend_label.setText(f"Blend: {int(100 - self.alpha*100)}% / {int(self.alpha*100)}%")
        h1, w1 = self.img1.shape[:2]
        h2, w2 = self.img2.shape[:2]
        max_w = max(w1, w2)
        max_h = max(h1, h2)
        canvas1 = np.zeros((max_h, max_w, 4), dtype=np.uint8)
        canvas2 = np.zeros((max_h, max_w, 4), dtype=np.uint8)
        y1_offset = (max_h - h1) // 2
        x1_offset = (max_w - w1) // 2
        canvas1[y1_offset:y1_offset+h1, x1_offset:x1_offset+w1] = self.img1
        y2_offset = (max_h - h2) // 2
        x2_offset = (max_w - w2) // 2
        canvas2[y2_offset:y2_offset+h2, x2_offset:x2_offset+w2] = self.img2
        blended = cv2.addWeighted(canvas1, 1 - self.alpha, canvas2, self.alpha, 0)
        height, width = blended.shape[:2]
        bytes_per_line = 4 * width
        q_img = QImage(blended.data, width, height, bytes_per_line, QImage.Format_RGBA8888).rgbSwapped()
        pixmap = QPixmap.fromImage(q_img)
        view_width = self.scroll_area.width() - 30
        view_height = self.scroll_area.height() - 30
        if pixmap.width() > view_width or pixmap.height() > view_height:
            scale_factor = min(view_width / pixmap.width(), view_height / pixmap.height())
            if scale_factor < 1:
                pixmap = pixmap.scaled(
                    int(pixmap.width() * scale_factor),
                    int(pixmap.height() * scale_factor),
                    Qt.KeepAspectRatio,
                    Qt.SmoothTransformation
                )
        self.image_label.setPixmap(pixmap)

    def resizeEvent(self, event):
        self.update_blend()
        super().resizeEvent(event)


class ImageProcessingApp(QMainWindow):
    """Main window for the advanced image editor."""
    def __init__(self):
        super().__init__()
        self.setWindowTitle("Advanced Image Editor")
        self.resize(1200, 800)
        self.images = []
        self.current_idx = -1
        self.crop_mode_active = False
        self.clipboard_transform = None
        self.show_previous_layer = False
        self.dark_ui = True
        self.init_ui()

    def init_ui(self):
        self.create_menu_bar()
        central_widget = QWidget()
        self.setCentralWidget(central_widget)
        main_layout = QVBoxLayout(central_widget)
        top_buttons = QHBoxLayout()
        self.btn_load = QPushButton("Load Images")
        self.btn_load.setMinimumHeight(36)
        self.btn_load.clicked.connect(self.load_images)
        top_buttons.addWidget(self.btn_load)
        self.btn_load_folder = QPushButton("Load Folder")
        self.btn_load_folder.setMinimumHeight(36)
        self.btn_load_folder.clicked.connect(self.load_folder)
        top_buttons.addWidget(self.btn_load_folder)
        self.btn_crop_mode = QPushButton("Toggle Crop Mode")
        self.btn_crop_mode.setMinimumHeight(36)
        self.btn_crop_mode.setCheckable(True)
        self.btn_crop_mode.clicked.connect(self.toggle_crop_mode)
        self.btn_crop_mode.setEnabled(False)
        top_buttons.addWidget(self.btn_crop_mode)
        self.btn_save_all = QPushButton("Save All")
        self.btn_save_all.setMinimumHeight(36)
        self.btn_save_all.clicked.connect(self.save_all_images)
        self.btn_save_all.setEnabled(False)
        top_buttons.addWidget(self.btn_save_all)
        main_layout.addLayout(top_buttons)
        splitter = QSplitter(Qt.Horizontal)
        left_panel = QWidget()
        left_layout = QVBoxLayout(left_panel)
        list_group = QGroupBox("Images")
        list_layout = QVBoxLayout(list_group)
        self.image_list = QListWidget()
        self.image_list.setSelectionMode(QListWidget.MultiSelection)
        self.image_list.setAlternatingRowColors(True)
        self.image_list.currentRowChanged.connect(self.image_selected)
        list_layout.addWidget(self.image_list)
        left_layout.addWidget(list_group)
        controls_group = QGroupBox("Image Transformations")
        controls_layout = QGridLayout(controls_group)
        controls_layout.addWidget(QLabel("Rotation:"), 0, 0)
        self.rotation_slider = QSlider(Qt.Horizontal)
        self.rotation_slider.setRange(-180, 180)
        self.rotation_slider.setValue(0)
        self.rotation_slider.setTickPosition(QSlider.TicksBelow)
        self.rotation_slider.setTickInterval(30)
        self.rotation_slider.valueChanged.connect(self.rotate_image)
        controls_layout.addWidget(self.rotation_slider, 0, 1)
        self.rotation_value = QLabel("0°")
        controls_layout.addWidget(self.rotation_value, 0, 2)
        controls_layout.addWidget(QLabel("Rotation Input:"), 1, 0)
        self.rotation_spinbox = QDoubleSpinBox()
        self.rotation_spinbox.setRange(-180.0, 180.0)
        self.rotation_spinbox.setDecimals(1)
        self.rotation_spinbox.setValue(0.0)
        self.rotation_spinbox.valueChanged.connect(self.sync_rotation_spinbox)
        controls_layout.addWidget(self.rotation_spinbox, 1, 1, 1, 2)
        fine_rotation_layout = QHBoxLayout()
        self.btn_rotate_left = QPushButton("← 0.5°")
        self.btn_rotate_left.clicked.connect(lambda: self.fine_rotate(-0.5))
        fine_rotation_layout.addWidget(self.btn_rotate_left)
        self.btn_rotate_right = QPushButton("0.5° →")
        self.btn_rotate_right.clicked.connect(lambda: self.fine_rotate(0.5))
        fine_rotation_layout.addWidget(self.btn_rotate_right)
        controls_layout.addLayout(fine_rotation_layout, 2, 0, 1, 3)
        controls_layout.addWidget(QLabel("X Position:"), 3, 0)
        self.x_translation = QSpinBox()
        self.x_translation.setRange(-1000, 1000)
        self.x_translation.setValue(0)
        self.x_translation.valueChanged.connect(self.apply_translation)
        controls_layout.addWidget(self.x_translation, 3, 1, 1, 2)
        controls_layout.addWidget(QLabel("Y Position:"), 4, 0)
        self.y_translation = QSpinBox()
        self.y_translation.setRange(-1000, 1000)
        self.y_translation.setValue(0)
        self.y_translation.valueChanged.connect(self.apply_translation)
        controls_layout.addWidget(self.y_translation, 4, 1, 1, 2)
        controls_layout.addWidget(QLabel("Transparency:"), 5, 0)
        self.transparency_slider = QSlider(Qt.Horizontal)
        self.transparency_slider.setRange(0, 255)
        self.transparency_slider.setValue(255)
        self.transparency_slider.setTickPosition(QSlider.TicksBelow)
        self.transparency_slider.setTickInterval(25)
        self.transparency_slider.valueChanged.connect(self.apply_transparency)
        controls_layout.addWidget(self.transparency_slider, 5, 1)
        self.transparency_value = QLabel("100%")
        controls_layout.addWidget(self.transparency_value, 5, 2)
        self.btn_clear_crop = QPushButton("Clear Crop")
        self.btn_clear_crop.clicked.connect(self.clear_crop)
        self.btn_clear_crop.setEnabled(False)
        controls_layout.addWidget(self.btn_clear_crop, 6, 0, 1, 3)
        self.btn_reset = QPushButton("Reset Current Image")
        self.btn_reset.clicked.connect(self.reset_current_image)
        self.btn_reset.setEnabled(False)
        controls_layout.addWidget(self.btn_reset, 7, 0, 1, 3)
        left_layout.addWidget(controls_group)
        self.progress_bar = QProgressBar()
        self.progress_bar.setStyleSheet("QProgressBar { text-align: center; }")
        self.progress_bar.setVisible(False)
        left_layout.addWidget(self.progress_bar)
        splitter.addWidget(left_panel)
        right_panel = QWidget()
        right_layout = QVBoxLayout(right_panel)
        preview_label = QLabel("Image Preview:")
        preview_label.setStyleSheet("font-weight: bold; margin: 5px;")
        right_layout.addWidget(preview_label)
        self.image_preview = ImagePreviewWidget()
        self.image_preview.cropChanged.connect(self.crop_changed)
        scroll_area = QScrollArea()
        scroll_area.setWidget(self.image_preview)
        scroll_area.setWidgetResizable(True)
        right_layout.addWidget(scroll_area)
        self.crop_hint_label = QLabel("Crop mode: Click and drag to define crop area")
        self.crop_hint_label.setStyleSheet("color: #FF6666; font-style: italic; padding: 5px;")
        self.crop_hint_label.setVisible(False)
        right_layout.addWidget(self.crop_hint_label)
        splitter.addWidget(right_panel)
        splitter.setSizes([int(self.width() * 0.3), int(self.width() * 0.7)])
        main_layout.addWidget(splitter)
        self.statusBar().setStyleSheet("QStatusBar { border-top: 1px solid #444; }")
        self.statusBar().showMessage("Ready – load images to begin editing")
        self.set_dark_theme()

    def create_menu_bar(self):
        menubar = QMenuBar(self)
        self.setMenuBar(menubar)
        file_menu = menubar.addMenu("&File")
        action_load_images = QAction("Load Images...", self)
        action_load_images.triggered.connect(self.load_images)
        file_menu.addAction(action_load_images)
        action_load_folder = QAction("Load Folder...", self)
        action_load_folder.triggered.connect(self.load_folder)
        file_menu.addAction(action_load_folder)
        file_menu.addSeparator()
        action_save_all = QAction("Save All...", self)
        action_save_all.triggered.connect(self.save_all_images)
        file_menu.addAction(action_save_all)
        file_menu.addSeparator()
        action_exit = QAction("Exit", self)
        action_exit.triggered.connect(self.close)
        file_menu.addAction(action_exit)
        edit_menu = menubar.addMenu("&Edit")
        action_reset = QAction("Reset Current Image", self)
        action_reset.triggered.connect(self.reset_current_image)
        edit_menu.addAction(action_reset)
        action_copy_transform = QAction("Copy Transform", self)
        action_copy_transform.triggered.connect(self.copy_transform)
        edit_menu.addAction(action_copy_transform)
        action_paste_transform = QAction("Paste Transform to Selected", self)
        action_paste_transform.triggered.connect(self.paste_transform)
        edit_menu.addAction(action_paste_transform)
        edit_menu.addSeparator()
        action_crop_all = QAction("Apply Crop to All", self)
        action_crop_all.triggered.connect(self.apply_crop_to_all)
        edit_menu.addAction(action_crop_all)
        action_crop_selected = QAction("Apply Crop to Selected", self)
        action_crop_selected.triggered.connect(self.apply_crop_to_selected)
        edit_menu.addAction(action_crop_selected)
        action_clear_crop = QAction("Clear Crop", self)
        action_clear_crop.triggered.connect(self.clear_crop)
        edit_menu.addAction(action_clear_crop)
        action_compare = QAction("Compare Images...", self)
        action_compare.triggered.connect(self.compare_images)
        edit_menu.addAction(action_compare)
        view_menu = menubar.addMenu("&View")
        theme_menu = view_menu.addMenu("UI Theme")
        action_dark = QAction("Dark UI", self, checkable=True)
        action_dark.setChecked(True)
        action_dark.triggered.connect(lambda: self.set_dark_theme())
        theme_menu.addAction(action_dark)
        action_light = QAction("Light UI", self, checkable=True)
        action_light.triggered.connect(lambda: self.set_light_theme())
        theme_menu.addAction(action_light)
        action_toggle_checkerboard = QAction("Show Transparency Grid", self, checkable=True)
        action_toggle_checkerboard.setChecked(True)
        action_toggle_checkerboard.triggered.connect(lambda checked: self.image_preview.toggle_checkerboard(checked))
        view_menu.addAction(action_toggle_checkerboard)
        action_toggle_prev_layer = QAction("Show Previous Layer", self, checkable=True)
        action_toggle_prev_layer.setChecked(False)
        action_toggle_prev_layer.triggered.connect(self.toggle_previous_layer)
        view_menu.addAction(action_toggle_prev_layer)
        help_menu = menubar.addMenu("&Help")
        action_about = QAction("About", self)
        action_about.triggered.connect(self.show_about)
        help_menu.addAction(action_about)

    def show_about(self):
        QMessageBox.information(self, "About", "Advanced Image Editor\nBuilt with PyQt5 and OpenCV.")

    def set_dark_theme(self):
        self.dark_ui = True
        dark_palette = QPalette()
        dark_color = QColor(45, 45, 45)
        text_color = QColor(210, 210, 210)
        dark_palette.setColor(QPalette.Window, dark_color)
        dark_palette.setColor(QPalette.WindowText, text_color)
        dark_palette.setColor(QPalette.Base, QColor(25, 25, 25))
        dark_palette.setColor(QPalette.AlternateBase, dark_color)
        dark_palette.setColor(QPalette.ToolTipBase, text_color)
        dark_palette.setColor(QPalette.ToolTipText, text_color)
        dark_palette.setColor(QPalette.Text, text_color)
        dark_palette.setColor(QPalette.Button, dark_color)
        dark_palette.setColor(QPalette.ButtonText, text_color)
        self.setPalette(dark_palette)
        self.setStyleSheet("""
            QMainWindow, QWidget { background-color: #2D2D2D; color: #D2D2D2; }
            QGroupBox { border: 1px solid #555; border-radius: 5px; margin-top: 10px; font-weight: bold; padding-top: 10px; }
            QGroupBox::title { subcontrol-origin: margin; left: 10px; padding: 0 3px; }
            QPushButton { background-color: #444; border: none; border-radius: 4px; padding: 5px 10px; }
            QPushButton:hover { background-color: #555; }
            QPushButton:pressed { background-color: #666; }
            QSlider::groove:horizontal { border: 1px solid #999; height: 6px; background: #444; margin: 2px 0; border-radius: 3px; }
            QSlider::handle:horizontal { background: #5A8CBF; border: 1px solid #5A8CBF; width: 14px; margin: -4px 0; border-radius: 7px; }
        """)
        self.statusBar().showMessage("Dark UI activated")

    def set_light_theme(self):
        self.dark_ui = False
        light_palette = QPalette()
        light_palette.setColor(QPalette.Window, QColor(240, 240, 240))
        light_palette.setColor(QPalette.WindowText, QColor(0, 0, 0))
        light_palette.setColor(QPalette.Base, QColor(255, 255, 255))
        light_palette.setColor(QPalette.AlternateBase, QColor(240, 240, 240))
        light_palette.setColor(QPalette.ToolTipBase, QColor(0, 0, 0))
        light_palette.setColor(QPalette.ToolTipText, QColor(0, 0, 0))
        light_palette.setColor(QPalette.Text, QColor(0, 0, 0))
        light_palette.setColor(QPalette.Button, QColor(200, 200, 200))
        light_palette.setColor(QPalette.ButtonText, QColor(0, 0, 0))
        self.setPalette(light_palette)
        self.setStyleSheet("""
            QMainWindow, QWidget { background-color: #F0F0F0; color: #000000; }
            QGroupBox { border: 1px solid #AAA; border-radius: 5px; margin-top: 10px; font-weight: bold; padding-top: 10px; }
            QGroupBox::title { subcontrol-origin: margin; left: 10px; padding: 0 3px; }
            QPushButton { background-color: #DDD; border: none; border-radius: 4px; padding: 5px 10px; }
            QPushButton:hover { background-color: #CCC; }
            QPushButton:pressed { background-color: #BBB; }
            QSlider::groove:horizontal { border: 1px solid #AAA; height: 6px; background: #DDD; margin: 2px 0; border-radius: 3px; }
            QSlider::handle:horizontal { background: #888; border: 1px solid #888; width: 14px; margin: -4px 0; border-radius: 7px; }
        """)
        self.statusBar().showMessage("Light UI activated")

    def load_images(self):
        files, _ = QFileDialog.getOpenFileNames(
            self, "Select Images", "", "Image Files (*.png *.jpg *.jpeg *.bmp *.tif *.tiff *.webp)"
        )
        if not files:
            return
        self._load_image_files(files)

    def load_folder(self):
        folder = QFileDialog.getExistingDirectory(self, "Select Folder with Images")
        if not folder:
            return
        file_list = os.listdir(folder)
        image_files = [os.path.join(folder, f) for f in file_list if f.lower().endswith(IMAGE_EXTENSIONS)]
        if not image_files:
            QMessageBox.warning(self, "No Images Found", "No valid image files found in the selected folder.")
            return
        self._load_image_files(image_files)

    def _load_image_files(self, files):
        self.images = []
        self.image_list.clear()
        self.current_idx = -1
        self.image_preview.set_image(None)
        failed_images = []
        self.progress_bar.setVisible(True)
        self.progress_bar.setRange(0, len(files))
        self.progress_bar.setValue(0)
        for i, file_path in enumerate(files):
            try:
                img_item = ImageItem(file_path)
                if img_item.is_valid():
                    self.images.append(img_item)
                    self.image_list.addItem(os.path.basename(file_path))
                else:
                    failed_images.append(file_path)
            except Exception as e:
                print(f"Failed to load {file_path}: {e}")
                failed_images.append(file_path)
            self.progress_bar.setValue(i + 1)
            QApplication.processEvents()
        self.progress_bar.setVisible(False)
        if self.images:
            self.image_list.setCurrentRow(0)
            self.btn_save_all.setEnabled(True)
            self.btn_crop_mode.setEnabled(True)
            self.statusBar().showMessage(f"Loaded {len(self.images)} images successfully")
        else:
            self.statusBar().showMessage("No valid images were loaded")
        if failed_images:
            failed_list = "\n".join(os.path.basename(f) for f in failed_images)
            QMessageBox.warning(self, "Some Images Failed to Load", f"Failed to load:\n{failed_list}")

    def fine_rotate(self, angle_delta):
        if self.current_idx < 0:
            return
        img_item = self.images[self.current_idx]
        new_angle = img_item.rotation_angle + angle_delta
        img_item.rotation_angle = new_angle
        self.rotation_slider.blockSignals(True)
        self.rotation_slider.setValue(int(new_angle))
        self.rotation_slider.blockSignals(False)
        self.rotation_spinbox.blockSignals(True)
        self.rotation_spinbox.setValue(new_angle)
        self.rotation_spinbox.blockSignals(False)
        self.rotation_value.setText(f"{new_angle:.1f}°")
        img_item.transformed = True
        self.apply_transformations()

    def rotate_image(self, angle):
        if self.current_idx < 0:
            return
        img_item = self.images[self.current_idx]
        img_item.rotation_angle = angle
        self.rotation_spinbox.blockSignals(True)
        self.rotation_spinbox.setValue(angle)
        self.rotation_spinbox.blockSignals(False)
        self.rotation_value.setText(f"{angle}°")
        img_item.transformed = True
        self.apply_transformations()

    def sync_rotation_spinbox(self, value):
        if self.current_idx < 0:
            return
        img_item = self.images[self.current_idx]
        img_item.rotation_angle = value
        self.rotation_slider.blockSignals(True)
        self.rotation_slider.setValue(int(value))
        self.rotation_slider.blockSignals(False)
        self.rotation_value.setText(f"{value:.1f}°")
        self.apply_transformations()

    def apply_translation(self):
        if self.current_idx < 0:
            return
        img_item = self.images[self.current_idx]
        img_item.x_offset = self.x_translation.value()
        img_item.y_offset = self.y_translation.value()
        img_item.transformed = True
        self.apply_transformations()

    def apply_transparency(self, value):
        if self.current_idx < 0:
            return
        img_item = self.images[self.current_idx]
        img_item.transparency = value
        percentage = int((value / 255) * 100)
        self.transparency_value.setText(f"{percentage}%")
        img_item.transformed = True
        self.apply_transformations()

    def apply_transformations(self):
        if self.current_idx < 0:
            return
        img_item = self.images[self.current_idx]
        if not img_item.is_valid():
            return
        img = img_item.original_img.copy()
        if img_item.rotation_angle != 0:
            (h, w) = img.shape[:2]
            center = (w // 2, h // 2)
            M = cv2.getRotationMatrix2D(center, img_item.rotation_angle, 1.0)
            cos = np.abs(M[0, 0])
            sin = np.abs(M[0, 1])
            new_w = int((h * sin) + (w * cos))
            new_h = int((h * cos) + (w * sin))
            M[0, 2] += (new_w / 2) - center[0]
            M[1, 2] += (new_h / 2) - center[1]
            img = cv2.warpAffine(img, M, (new_w, new_h), flags=cv2.INTER_LINEAR,
                                 borderMode=cv2.BORDER_CONSTANT, borderValue=(0, 0, 0, 0))
        if img_item.x_offset != 0 or img_item.y_offset != 0:
            (h, w) = img.shape[:2]
            M = np.float32([[1, 0, img_item.x_offset], [0, 1, img_item.y_offset]])
            img = cv2.warpAffine(img, M, (w, h),
                                 borderMode=cv2.BORDER_CONSTANT, borderValue=(0, 0, 0, 0))
        if img_item.transparency < 255:
            (b, g, r, a) = cv2.split(img)
            a = cv2.multiply(a, img_item.transparency / 255.0)
            img = cv2.merge([b, g, r, a.astype(np.uint8)])
        img_item.display_img = img
        if img_item.crop_rect is not None and not self.crop_mode_active:
            x, y, w_crop, h_crop = img_item.crop_rect
            x = max(0, min(x, img.shape[1] - 1))
            y = max(0, min(y, img.shape[0] - 1))
            w_crop = min(w_crop, img.shape[1] - x)
            h_crop = min(h_crop, img.shape[0] - y)
            cropped = img[y:y+h_crop, x:x+w_crop]
            self.image_preview.set_image(cropped)
        else:
            self.image_preview.set_image(img)
        self.update_background_layer()

    def update_background_layer(self):
        if self.show_previous_layer and self.current_idx > 0:
            prev_item = self.images[self.current_idx - 1]
            if prev_item.display_img is not None:
                self.image_preview.set_background_image(prev_item.display_img)
            else:
                self.image_preview.set_background_image(None)
        else:
            self.image_preview.set_background_image(None)

    def reset_current_image(self):
        if self.current_idx < 0:
            return
        img_item = self.images[self.current_idx]
        img_item.reset()
        self.rotation_slider.blockSignals(True)
        self.rotation_slider.setValue(0)
        self.rotation_slider.blockSignals(False)
        self.rotation_spinbox.blockSignals(True)
        self.rotation_spinbox.setValue(0.0)
        self.rotation_spinbox.blockSignals(False)
        self.rotation_value.setText("0°")
        self.x_translation.blockSignals(True)
        self.x_translation.setValue(0)
        self.x_translation.blockSignals(False)
        self.y_translation.blockSignals(True)
        self.y_translation.setValue(0)
        self.y_translation.blockSignals(False)
        self.transparency_slider.blockSignals(True)
        self.transparency_slider.setValue(255)
        self.transparency_slider.blockSignals(False)
        self.transparency_value.setText("100%")
        self.image_preview.set_image(img_item.original_img)
        self.statusBar().showMessage("Current image reset")

    def image_selected(self, index):
        if index < 0 or index >= len(self.images):
            self.current_idx = -1
            self.image_preview.set_image(None)
            self.btn_reset.setEnabled(False)
            self.btn_clear_crop.setEnabled(False)
            return
        self.current_idx = index
        img_item = self.images[index]
        self.rotation_slider.blockSignals(True)
        self.rotation_slider.setValue(int(img_item.rotation_angle))
        self.rotation_slider.blockSignals(False)
        self.rotation_spinbox.blockSignals(True)
        self.rotation_spinbox.setValue(img_item.rotation_angle)
        self.rotation_spinbox.blockSignals(False)
        self.rotation_value.setText(f"{img_item.rotation_angle:.1f}°")
        self.x_translation.blockSignals(True)
        self.x_translation.setValue(img_item.x_offset)
        self.x_translation.blockSignals(False)
        self.y_translation.blockSignals(True)
        self.y_translation.setValue(img_item.y_offset)
        self.y_translation.blockSignals(False)
        self.transparency_slider.blockSignals(True)
        self.transparency_slider.setValue(img_item.transparency)
        self.transparency_slider.blockSignals(False)
        percentage = int((img_item.transparency / 255) * 100)
        self.transparency_value.setText(f"{percentage}%")
        if not self.crop_mode_active and img_item.crop_rect:
            x, y, w, h = img_item.crop_rect
            self.image_preview.crop_rect = QRect(x, y, w, h)
        else:
            self.image_preview.crop_rect = None
        self.btn_reset.setEnabled(True)
        self.btn_clear_crop.setEnabled(img_item.crop_rect is not None)
        self.apply_transformations()

    def toggle_crop_mode(self, checked):
        self.crop_mode_active = checked
        self.image_preview.set_crop_mode(checked)
        self.crop_hint_label.setVisible(checked)
        self.rotation_slider.setEnabled(not checked)
        self.rotation_spinbox.setEnabled(not checked)
        self.btn_rotate_left.setEnabled(not checked)
        self.btn_rotate_right.setEnabled(not checked)
        self.x_translation.setEnabled(not checked)
        self.y_translation.setEnabled(not checked)
        self.transparency_slider.setEnabled(not checked)
        if checked:
            self.btn_crop_mode.setText("Cancel Crop Mode")
            if self.current_idx >= 0:
                # Prepare for a new crop; do not clear existing crop rect.
                self.image_preview.crop_rect = None
                self.image_preview.set_image(self.images[self.current_idx].display_img)
        else:
            self.btn_crop_mode.setText("Toggle Crop Mode")
            if self.current_idx >= 0:
                img_item = self.images[self.current_idx]
                if img_item.crop_rect:
                    x, y, w, h = img_item.crop_rect
                    self.image_preview.crop_rect = QRect(x, y, w, h)
                self.apply_transformations()
        self.btn_clear_crop.setEnabled(
            self.current_idx >= 0 and self.images[self.current_idx].crop_rect is not None
        )

    def crop_changed(self, rect):
        if self.current_idx < 0:
            return
        img_item = self.images[self.current_idx]
        img_item.crop_rect = (rect.x(), rect.y(), rect.width(), rect.height())
        self.btn_clear_crop.setEnabled(True)
        self.btn_crop_mode.setChecked(False)
        self.toggle_crop_mode(False)
        self.statusBar().showMessage(f"Crop area defined: {rect.width()}x{rect.height()} px")

    def apply_crop_to_all(self):
        if self.current_idx < 0:
            return
        current_crop = self.images[self.current_idx].crop_rect
        if current_crop is None:
            self.statusBar().showMessage("No crop area defined in current image")
            return
        for img_item in self.images:
            img_item.crop_rect = current_crop
        self.btn_crop_mode.setChecked(False)
        self.toggle_crop_mode(False)
        self.statusBar().showMessage("Crop applied to all images")
        self.image_selected(self.current_idx)

    def apply_crop_to_selected(self):
        if self.current_idx < 0:
            return
        current_crop = self.images[self.current_idx].crop_rect
        if current_crop is None:
            self.statusBar().showMessage("No crop area defined in current image")
            return
        selected = self.image_list.selectedIndexes()
        if not selected:
            self.statusBar().showMessage("No images selected; crop not applied")
            return
        for index in selected:
            idx = index.row()
            self.images[idx].crop_rect = current_crop
        self.statusBar().showMessage("Crop applied to selected images")

    def clear_crop(self):
        if self.current_idx < 0:
            return
        img_item = self.images[self.current_idx]
        img_item.crop_rect = None
        self.image_preview.crop_rect = None
        self.apply_transformations()
        self.statusBar().showMessage("Crop cleared")

    def copy_transform(self):
        if self.current_idx < 0:
            return
        img_item = self.images[self.current_idx]
        self.clipboard_transform = {
            "rotation_angle": img_item.rotation_angle,
            "x_offset": img_item.x_offset,
            "y_offset": img_item.y_offset,
            "transparency": img_item.transparency,
            "crop_rect": img_item.crop_rect
        }
        self.statusBar().showMessage("Transformations copied to clipboard")

    def paste_transform(self):
        selected = self.image_list.selectedIndexes()
        indices = [idx.row() for idx in selected] if selected else [self.current_idx]
        if not indices or self.clipboard_transform is None:
            self.statusBar().showMessage("No transformations in clipboard to paste")
            return
        for idx in indices:
            img_item = self.images[idx]
            img_item.rotation_angle = self.clipboard_transform.get("rotation_angle", 0)
            img_item.x_offset = self.clipboard_transform.get("x_offset", 0)
            img_item.y_offset = self.clipboard_transform.get("y_offset", 0)
            img_item.transparency = self.clipboard_transform.get("transparency", 255)
            img_item.crop_rect = self.clipboard_transform.get("crop_rect", None)
        self.image_selected(self.current_idx)
        self.statusBar().showMessage("Transformations pasted to selected images")

    def toggle_previous_layer(self, checked):
        self.show_previous_layer = checked
        self.update_background_layer()

    def compare_images(self):
        selected = self.image_list.selectedIndexes()
        if len(selected) != 2:
            QMessageBox.warning(self, "Compare Images", "Please select exactly two images to compare.")
            return
        idx1, idx2 = selected[0].row(), selected[1].row()
        img1 = self.images[idx1].display_img
        img2 = self.images[idx2].display_img
        if img1 is None or img2 is None:
            QMessageBox.warning(self, "Compare Images", "One or both selected images are invalid.")
            return
        dialog = CompareDialog(img1, img2, self)
        dialog.exec_()

    def update_background_layer(self):
        if self.show_previous_layer and self.current_idx > 0:
            prev_item = self.images[self.current_idx - 1]
            if prev_item.display_img is not None:
                self.image_preview.set_background_image(prev_item.display_img)
            else:
                self.image_preview.set_background_image(None)
        else:
            self.image_preview.set_background_image(None)

    def save_all_images(self):
        if not self.images:
            return
        save_dir = QFileDialog.getExistingDirectory(self, "Select Directory to Save Images")
        if not save_dir:
            return
        for img_item in self.images:
            if not img_item.is_valid():
                continue
            img = img_item.original_img.copy()
            if img_item.rotation_angle != 0:
                (h, w) = img.shape[:2]
                center = (w // 2, h // 2)
                M = cv2.getRotationMatrix2D(center, img_item.rotation_angle, 1.0)
                cos = np.abs(M[0, 0])
                sin = np.abs(M[0, 1])
                new_w = int((h * sin) + (w * cos))
                new_h = int((h * cos) + (w * sin))
                M[0, 2] += (new_w / 2) - center[0]
                M[1, 2] += (new_h / 2) - center[1]
                img = cv2.warpAffine(img, M, (new_w, new_h), flags=cv2.INTER_LINEAR,
                                     borderMode=cv2.BORDER_CONSTANT, borderValue=(0, 0, 0, 0))
            if img_item.x_offset != 0 or img_item.y_offset != 0:
                (h, w) = img.shape[:2]
                M = np.float32([[1, 0, img_item.x_offset], [0, 1, img_item.y_offset]])
                img = cv2.warpAffine(img, M, (w, h),
                                     borderMode=cv2.BORDER_CONSTANT, borderValue=(0, 0, 0, 0))
            if img_item.transparency < 255:
                (b, g, r, a) = cv2.split(img)
                a = cv2.multiply(a, img_item.transparency / 255.0)
                img = cv2.merge([b, g, r, a.astype(np.uint8)])
            if img_item.crop_rect is not None:
                x, y, w_crop, h_crop = img_item.crop_rect
                x = max(0, min(x, img.shape[1]-1))
                y = max(0, min(y, img.shape[0]-1))
                w_crop = min(w_crop, img.shape[1]-x)
                h_crop = min(h_crop, img.shape[0]-y)
                img = img[y:y+h_crop, x:x+w_crop]
            filename, ext = os.path.splitext(img_item.filename)
            save_path = os.path.join(save_dir, f"{filename}_edited{ext}")
            cv2.imwrite(save_path, img)
        self.statusBar().showMessage(f"Saved {len(self.images)} images to {save_dir}")

if __name__ == '__main__':
    app = QApplication(sys.argv)
    window = ImageProcessingApp()
    window.show()
    sys.exit(app.exec_())


SystemExit: 0

C:\Users\antoi\AppData\Roaming\Python\Python312\site-packages\IPython\core\interactiveshell.py:3585: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
